<a href="https://colab.research.google.com/github/Athugodage/RuLawSimplification/blob/main/trained_models/t5_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Athugodage/RuLawSimplification.git

Cloning into 'RuLawSimplification'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 142 (delta 24), reused 30 (delta 8), pack-reused 76
Receiving objects: 100% (142/142), 43.30 MiB | 16.41 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [ ]:
import json

with open('/content/RuLawSimplification/two_way/two_way.json', 'r', encoding='utf8') as rp:
    a = json.load(rp)

In [ ]:
a[100]

In [ ]:
import pandas as pd

df = pd.DataFrame({'Text': [], 'Comment': []})


for article in a:
    for sujet in article['article text']:
        df = df.append({'Text': sujet['text'], 'Comment': sujet['comment']}, ignore_index=True)


In [ ]:
df

In [ ]:
df.to_csv('two_way_trial.csv', index=False)

In [ ]:
df.shape

In [ ]:
df.iloc[:2500].to_csv('train.csv', index=False, header=None)
df.iloc[2500:2800].to_csv('validation.csv', index=False, header=None)
df.iloc[2800:].to_csv('test.csv', index=False, header=None)

In [2]:
!pip install Sentencepiece
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 27.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 32.7 MB/s 
     |████████████████████████████████| 6.6 MB 58.9 MB/s 
     |████████████████████████████████| 120 kB 68.7 MB/s 


In [3]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [5]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 12.2 MB/s 
     |████████████████████████████████| 115 kB 51.8 MB/s 
     |████████████████████████████████| 212 kB 48.5 MB/s 
     |████████████████████████████████| 127 kB 27.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
pd.read_csv('test.csv', encoding='utf8', sep=',', index_col=False, header=None)

In [6]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from datasets import load_dataset


dataset = load_dataset('csv', 
                       data_files={'train': ['train.csv'],
                                   'validation': ['validation.csv'],
                                   'test': ['test.csv']},
                       encoding='utf8',
                       index_col = False,
                       header=None,
                       sep=','
                      )

In [ ]:
print(dataset['train'])
print(dataset['validation'])
print(dataset['test'])

In [ ]:
dataset.shape

In [ ]:
dataset.push_to_hub('marcus2000/twowaydata')

In [7]:
from datasets import load_dataset

dataset = load_dataset('marcus2000/twowaydata')

Downloading:   0%|          | 0.00/892 [00:00<?, ?B/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/marcus2000___parquet/marcus2000--twowaydata-70c3f9955e1f2e89/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset['train']

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"], 2)

In [8]:
from transformers import AutoTokenizer, AutoModel, AutoConfig


tokenizer = AutoTokenizer.from_pretrained("t5-base")

# "bert-base-cased"
# DeepPavlov/rubert-base-cased-conversational


/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [9]:
max_input_length = 100
max_target_length = 100


def preprocess_function(examples):
    inputs = [ex for ex in examples['0']]  # 0 - Текст
    targets = [ex for ex in examples['1']]  # 1 - Комментарий
    model_inputs = tokenizer(inputs, 
                             max_length=max_input_length,
                             truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
tokenized_datasets

DatasetDict({
    validation: Dataset({
        features: ['0', '1', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 300
    })
    train: Dataset({
        features: ['0', '1', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2500
    })
    test: Dataset({
        features: ['0', '1', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 492
    })
})

In [12]:
for x in tokenized_datasets['train']:
    print(x['0'], x['input_ids'])

Указанное понимание не соотносится и с буквальным смыслом абзаца второго пункта 1 статьи 134 Закона о банкротстве, закрепляющего единственный критерий внеочередного удовлетворения требований кредитора - направленность платежей на предотвращение катастроф либо гибели людей, вне зависимости от того, связано ли обязательство по внесению платежей с исполнением обязанностей по договору, заключенному в рамках обычной хозяйственной деятельности должника, или нет. [3, 2, 12095, 2, 25873, 9890, 1757, 3, 2, 2044, 14391, 6469, 27616, 1757, 3, 14142, 24697, 17059, 9890, 5345, 22682, 5345, 2, 3, 2795, 12681, 3, 2, 3700, 6652, 16624, 6588, 2, 7184, 2, 6469, 12681, 6469, 2, 5345, 21044, 6469, 3, 2533, 2, 2533, 2, 2533, 8724, 9592, 10338, 2, 2044, 3, 2, 3700, 7184, 6652, 15517, 209, 3, 10458, 22581, 2, 2795, 3, 23747, 3, 2, 2533, 12377, 8194, 3, 2044, 3, 2, 25873, 6652, 10338, 6725, 24832, 1757, 6, 3, 2, 2533, 6652, 13400, 2, 6588, 2, 1757, 2, 2044, 3, 1757, 5814, 22420, 24832, 1]
3. Абзац второй пунк

In [13]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

In [14]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

batch_size = 16

args = Seq2SeqTrainingArguments("fine_tuned_t5_model",
    evaluation_strategy = "epoch",
    learning_rate=2e-3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    push_to_hub=True,
    predict_with_generate=True,
    fp16=True
)

In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [17]:
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 116 kB 27.2 MB/s 


In [18]:
from datasets import load_metric


metric = load_metric("sacrebleu")

In [19]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [20]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

Cloning https://huggingface.co/marcus2000/fine_tuned_t5_model into local empty directory.
Using cuda_amp half precision backend


In [21]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: 1, 0. If 1, 0 are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2500
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1570


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,1.956605,0.007400,19.000000
2,No log,1.833036,0.026000,19.000000
3,No log,1.795733,0.040500,19.000000
4,1.911500,1.744074,0.007300,19.000000
5,1.911500,1.787454,0.033000,19.000000
6,1.911500,2.041889,0.009300,19.000000
7,1.506600,2.074975,0.009900,19.000000
8,1.506600,2.074361,0.009900,19.000000
9,1.506600,2.073804,0.009900,19.000000
10,1.616300,2.073630,0.009900,19.000000


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: 1, 0. If 1, 0 are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: 1, 0. If 1, 0 are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: 1, 0. If 1, 0 are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 16
Saving model checkpoint to fine_tu

TrainOutput(global_step=1570, training_loss=1.6762330996762416, metrics={'train_runtime': 682.5349, 'train_samples_per_second': 36.628, 'train_steps_per_second': 2.3, 'total_flos': 2973427200000000.0, 'train_loss': 1.6762330996762416, 'epoch': 10.0})

In [22]:
predidcted = trainer.predict(tokenized_datasets['test'])

The following columns in the test set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: 1, 0. If 1, 0 are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 492
  Batch size = 16


In [23]:
predidcted

PredictionOutput(predictions=array([[    0,     3,     2, ..., 25083,     3,     2],
       [    0,     3,     2, ..., 25083,     3,     2],
       [    0,     3,     2, ..., 25083,     3,     2],
       ...,
       [    0,     3,     2, ..., 25083,     3,     2],
       [    0,     3,     2, ..., 25083,     3,     2],
       [    0,     3,     2, ..., 15517,  8724, 10458]]), label_ids=array([[    3,     2,  2533, ...,     2,  2795,     1],
       [    3,     2,  5345, ...,  -100,  -100,  -100],
       [    3,     2,  1757, ...,     2,     3,     1],
       ...,
       [    3,     2, 25752, ...,     3,  5814,     1],
       [    3,     2,  1757, ...,     2,  2044,     1],
       [    3,     2,  2044, ..., 26672,     2,     1]]), metrics={'test_loss': 2.114356517791748, 'test_bleu': 0.0101, 'test_gen_len': 19.0, 'test_runtime': 19.3313, 'test_samples_per_second': 25.451, 'test_steps_per_second': 1.604})

In [24]:
trainer.push_to_hub()

Saving model checkpoint to fine_tuned_t5_model
Configuration saved in fine_tuned_t5_model/config.json
Model weights saved in fine_tuned_t5_model/pytorch_model.bin
tokenizer config file saved in fine_tuned_t5_model/tokenizer_config.json
Special tokens file saved in fine_tuned_t5_model/special_tokens_map.json
Copy vocab file to fine_tuned_t5_model/spiece.model
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/850M [00:00<?, ?B/s]

Upload file runs/Sep08_19-16-47_099a2053a514/events.out.tfevents.1662664658.099a2053a514.71.0:  38%|###7      …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/marcus2000/fine_tuned_t5_model
   a950581..a1fd0af  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/marcus2000/fine_tuned_t5_model
   a950581..a1fd0af  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'metrics': [{'name': 'Bleu', 'type': 'bleu', 'value': 0.0099}]}
To https://huggingface.co/marcus2000/fine_tuned_t5_model
   a1fd0af..be808e3  main -> main

   a1fd0af..be808e3  main -> main



'https://huggingface.co/marcus2000/fine_tuned_t5_model/commit/a1fd0af0647f2f12671d0978ad351d47f56a0d82'

In [25]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 5.2 MB/s 


In [26]:
import evaluate
# evaluate.list_evaluation_modules(module_type='metric', with_details=True)

In [27]:
metric_value = 8.1  # value obtained from https://catalog.ngc.nvidia.com/orgs/nvidia/teams/nemo/models/stt_en_conformer_ctc_small 

evaluate.push_to_hub(
    model_id='marcus2000/fine_tuned_t5_model',
    task_type="automatic-speech-recognition",
    dataset_type="librispeech_asr",
    dataset_name="Librispeech (clean)",
    metric_type="wer",
    metric_name="WER",
    dataset_split="test",  # corresponds to test-clean set
    dataset_config="other",  # corresponds to test-clean set
    dataset_args=dict(language="en"),  # metadata for dataset
    # the actual score obtained by the model
    metric_value=metric_value,
)

Downloading:   0%|          | 0.00/2.01k [00:00<?, ?B/s]

'https://huggingface.co/marcus2000/fine_tuned_t5_model/blob/main/README.md'